# Build the X datasets for each type of bet

In [1]:
# initializes the datasets for each type of bet
import pandas as pd

# function creates n number of generic column names
def generate_col_names(num_needed: int) -> list:
    return ['Column' + str(i+1) for i in range(num_needed)]

col_names = generate_col_names(1)
pl_X_df = pd.DataFrame(columns=col_names)

col_names = generate_col_names(502)
ml_X_df = pd.DataFrame(columns=col_names)

col_names = generate_col_names(1)
ou_X_df = pd.DataFrame()

In [6]:
pwd

'c:\\Users\\Owner\\Desktop\\cs stuff\\Open Source\\nn-nhl\\scripts'

In [8]:
# generates a list of every my_id for every game from the start to the end year
import sys  
import os
sys.path.insert(0, 'C:/Users/Owner/Desktop/cs_stuff/Open Source/nn-nhl/scripts/')

from getter_methods import get_three_letter_code

every_my_id = list()
start_year = 18
end_year = 23
for year in range(start_year, end_year):
    print(os.getcwd())
    file_path = f"data/adv_20{year}.csv"
    df = pd.read_csv(file_path)

    num_rows = len(df)
    for i in range(num_rows):
        team_abv = get_three_letter_code(df['Team'].iloc[i])
        date = df['Date'].iloc[i]
        every_my_id.append(f"{team_abv}-{date}")

FileNotFoundError: [Errno 2] No such file or directory: 'nn-nhl/data/2022odds.csv'

In [ ]:
from getter_methods import get_last_n, get_rest_days, get_year

# this function returns the data to be used in a single row for all the adv metrics
def get_main_metrics(my_id: str) -> list:
    to_return = []
    to_return.extend(get_last_n(my_id, 1))
    to_return.extend(get_last_n(my_id, 3))
    to_return.extend(get_last_n(my_id, 5))
    to_return.extend(get_last_n(my_id, 10))
    to_return.extend([get_rest_days(my_id)])
    to_return.extend([get_year(my_id)])
    return to_return

In [ ]:
from getter_methods import get_open_ml, get_close_ml

# this loop creates the full X dataset for the ml
ml_index = 0

num_of_ids = len(every_my_id)
for i in range(0, num_of_ids, 2):
    try:
        team1_id = every_my_id[i]
        team2_id = every_my_id[i + 1]

        team1_metrics = get_main_metrics(team1_id)
        team2_metrics = get_main_metrics(team2_id)

        team1_open_ml = get_open_ml(team1_id)
        team2_open_ml = get_open_ml(team2_id)
        team1_close_ml = get_close_ml(team1_id)
        team2_close_ml = get_close_ml(team2_id)

        ml_row_1 = team1_metrics + team2_metrics + team1_open_ml
        ml_row_2 = team1_metrics + team2_metrics + team1_close_ml
        ml_row_3 = team2_metrics + team1_metrics + team2_open_ml
        ml_row_4 = team2_metrics + team1_metrics + team2_close_ml

        ml_X_df.loc[ml_index] = ml_row_1
        ml_index += 1
        ml_X_df.loc[ml_index] = ml_row_2
        ml_index += 1
        ml_X_df.loc[ml_index] = ml_row_3
        ml_index += 1
        ml_X_df.loc[ml_index] = ml_row_4
        ml_index += 1
    except:
        pass

    print(f"{int(i / 2)}/{int(num_of_ids / 2)}")

ml_X_df = ml_X_df.dropna()
ml_X_df.to_csv("ml_X.csv")

In [ ]:
from getter_methods import get_puck_line, get_puck_line_odds

pl_index = 0

num_of_ids = len(every_my_id)
for i in range(0, num_of_ids, 2):
    try:
        team1_id = every_my_id[i]
        team2_id = every_my_id[i + 1]

        team1_metrics = get_main_metrics(team1_id)
        team2_metrics = get_main_metrics(team2_id)

        team1_odds = get_puck_line_odds(team1_id)
        team2_odds = get_puck_line_odds(team2_id)

        team1_line = get_puck_line(team1_id)
        team2_line = get_puck_line(team2_id)

        pl_row_1 = team1_metrics + team2_metrics + team1_odds + team1_line
        pl_row_2 = team1_metrics + team2_metrics + team2_odds + team2_line
        print(len(pl_row_1))

        pl_X_df.loc[pl_index] = pl_row_1
        pl_index += 1
        pl_X_df.loc[pl_index] = pl_row_2
        pl_index += 1
    except:
        pass

    print(f"{int(i / 2)}/{int(num_of_ids / 2)}")

pl_X_df = pl_X_df.dropna()
pl_X_df.to_csv("pl_X.csv")